In [1]:
#Import libaries
import os
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
#opress warnings
import warnings
warnings.filterwarnings("ignore")
from shapely.geometry import LineString, Point
import math
import geopandas as gpd
from shapely import wkt

In [2]:
#load cleaned road data
roads = pd.read_csv('../data/raw/_roads3.csv')

In [3]:
def lookup_coordinate(lrp, road_name,search_lon):
    """
    This function looks up the coordinates of a given lrp and road name.
    """
    mask = (roads['lrp'] == lrp) & (roads['road'] == road_name)
    matching_rows = roads[mask]
    
    if not matching_rows.empty:
        if search_lon:
            return matching_rows.iloc[0]['lon']
        else:
            return matching_rows.iloc[0]['lat']
    else:
        return None


In [4]:
def create_linestring(row):
    """
    Creates a linestring from the start and end point of a row
    """
    start_point = (row['lon_start'], row['lat_start'])
    end_point = (row['lon_end'], row['lat_end'])
    return LineString([start_point, end_point])

In [5]:
def clean_road_data(path):
    """
    This function cleans the road data and returns a dataframe with the cleaned data.
    """
    
    #open file
    file = open(path, 'r')
    soup = BeautifulSoup(file)
    #Search Data Table
    table = soup.find('table', attrs={'class':'clsTbl'})
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    data = []
    #loop through rows
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) 
    df_clean = pd.DataFrame(data)
    #drop first 4 row without data
    df = df_clean.drop(df_clean.index[0:4])

    #drop last 2 row without data
    df = df.drop(df.index[-2:])
    #make first row to header
    new_header = df.iloc[0] 
    #take the data less the header row
    df = df[1:]
    df.columns = new_header
    #store headers in list
    headers = df.columns.values.tolist()
    #drop last 2 elements of headers list
    headers = headers[:-2]
    #add 'link_no', 'name' to headers
    headers.insert(0, 'link_no')
    headers.insert(1, 'name')
    #rename headers to headers
    df.columns = headers
    #remove columns with None header    
    df = df.dropna(axis=1, how='all')
    #rename headers
    headers = df.columns.values.tolist()
    headers[2] = 'LRP_Start'
    headers[3] = 'Offset_Start'
    headers[4] = 'Chainage_Start'
    headers[5] = 'LRP_End'
    headers[6] = 'Offset_End'
    headers[7] = 'Chainage_End'
    
    df_final = df
    #set final headers
    df_final.columns = headers
    #remove rows with 'NS' in 'Heavy Truck' column
    df_final = df_final[df_final['Heavy Truck'] != 'NS']
    
   
    return df_final
    

In [6]:
#remove road_data.csv if exists
if os.path.isfile('../data/processed/road_data.csv'):
    os.remove('../data/processed/road_data.csv')
#loop through all files in data folder that start with N
for file in os.listdir('../data/raw/road_data'):
    if file.startswith('N'):
        #define road_name
        road_name = file.split('.')[0]        
        print("Parsing: "+road_name)
        #skip N203 because no data
        if road_name == 'N203':
            continue
        #read data            
        df = clean_road_data('../data/raw/road_data/'+file)
        #if dataframe is empty or only one entry, skip
        
        #keep only first occurence if link_no is duplicated
        df = df.drop_duplicates(subset='link_no', keep='first')
        if df.empty or len(df) == 1:
            continue
        
        
        
        #add road name column
        df['road_name'] = road_name
        #add coordinates
        df['lon_start'] = df.apply(lambda row: lookup_coordinate(row['LRP_Start'], row['road_name'],True), axis=1)
        df['lat_start'] = df.apply(lambda row: lookup_coordinate(row['LRP_Start'], row['road_name'],False), axis=1)
        df['lon_end'] = df.apply(lambda row: lookup_coordinate(row['LRP_End'], row['road_name'],True), axis=1)
        df['lat_end'] = df.apply(lambda row: lookup_coordinate(row['LRP_End'], row['road_name'],False), axis=1)
        #skip if coordinates are None
        if df['lon_start'].isnull().values.any() or df['lat_start'].isnull().values.any() or df['lon_end'].isnull().values.any() or df['lat_end'].isnull().values.any():
            continue
        #find mid point for water stations
        df['lon_mid'] = (df['lon_start'] + df['lon_end'])/2
        df['lat_mid'] = (df['lat_start'] + df['lat_end'])/2
        df['mid_geometry'] = df.apply(lambda row: Point(row['lon_mid'], row['lat_mid']), axis=1)
        
        # Create a new 'geometry' column containing LINESTRING objects
        df['geometry'] = df.apply(create_linestring, axis=1)
        
        #write to global csv if exists
        #catch error if cannot write to csv
        try:
            if os.path.isfile('../data/processed/road_data.csv'):
                df.to_csv('../data/processed/road_data.csv', mode='a', header=False,index=False)
            #else create new file
            else:
                df.to_csv('../data/processed/road_data.csv', mode='a', header=True,index=False)
        except:
            print("Could not write to csv")
            pass
        

Parsing: N501
Parsing: N604
Parsing: N101
Parsing: N204
Parsing: N508
Parsing: N211
Parsing: N303
Parsing: N703
Parsing: N108
Parsing: N2
Parsing: N406
Parsing: N514
Parsing: N126
Parsing: N111
Parsing: N123
Parsing: N208
Parsing: N511
Parsing: N403
Parsing: N7
Parsing: N706
Parsing: N713
Parsing: N504
Parsing: N518
Parsing: N802
Parsing: N104
Parsing: N801
Parsing: N107
Parsing: N129
Parsing: N507
Parsing: N602
Parsing: N710
Parsing: N808
Parsing: N705
Parsing: N120
Parsing: N4
Parsing: N520
Parsing: N112
Parsing: N405
Parsing: N1
Parsing: N125
Parsing: N517
Parsing: N212
Parsing: N207
Parsing: N8
Parsing: N102
Parsing: N804
Parsing: N540
Parsing: N709
Parsing: N715
Parsing: N309
Parsing: N502
Parsing: N519
Parsing: N105
Parsing: N803
Parsing: N712
Parsing: N119
Parsing: N505
Parsing: N6
Parsing: N402
Parsing: N510
Parsing: N209
Parsing: N707
Parsing: N110
Parsing: N109
Parsing: N702
Parsing: N127
Parsing: N515
Parsing: N407
Parsing: N3
Parsing: N302
Parsing: N210
Parsing: N509
Parsin

<hr>

<h1>Vulnerablity </h1>

<h2>Water Data</h2>

In [7]:
road_segments = pd.read_csv('../data/processed/road_data.csv')
LRP_data = pd.read_csv('../data/raw/_roads3.csv')
water_data = pd.read_excel('../data/raw/flood_data/geolocations_stations.xlsx')

In [8]:
def excess_water(segment):
    if math.isnan(segment['Highest Water Level']):
        return segment['Water Level ']-segment['Danger Level ']
    else:
        return segment['Highest Water Level']-segment['Danger Level ']



In [9]:

# Create Excess Water Level
water_data['Excess Water Level'] = water_data.apply(excess_water,axis=1)
water_data['geometry'] = water_data.apply(lambda row: Point(row['Longitude '], row['Latitude ']), axis=1)

In [10]:

#road_segments to gdf
road_segments['mid_geometry'] = road_segments['mid_geometry'].astype(str).apply(wkt.loads)

road_segments = gpd.GeoDataFrame(road_segments, geometry='mid_geometry')
#water_data to gdf

water_data = gpd.GeoDataFrame(water_data, geometry='geometry')
# conduct Spatial Join using Nearest Neighbour
joined = gpd.sjoin_nearest(road_segments, water_data, how="inner", lsuffix="")

road_segments_water = joined.drop(['index_right', 'Station Name ', 'River Name ', 'Division ',
       'District ', 'Upazilla ', 'Union ', 'Average Land Level ', 
       'Unnamed: 8', 'Longitude ', 'Latitude '], axis=1)



<h2>Bridges</h2>

In [11]:
#load bridges
bridges = pd.read_excel('../data/raw/BMMS_overview.xlsx')

In [12]:
def find_road_segment_of_bridge(bridge):
    """Find road segment of bridge
    
    Parameters
    ----------
    bridge : pandas.Series
        Series containing bridge data
    Returns
    -------
    string
        link_no of road segment
    """
    
    bridge_chainage = bridge['km']
    bridge_road_name = bridge['road']
    mask = (bridge_chainage >= road_segments['Chainage_Start'])&(bridge_chainage <= road_segments['Chainage_End']) & (road_segments['road_name'] == bridge_road_name)
    matching_rows = road_segments[mask]
    
    if not matching_rows.empty:        
            return matching_rows.iloc[0]['link_no']
        
    else:
        return None

In [13]:
bridges['link_no'] = bridges.apply(find_road_segment_of_bridge, axis=1)

In [14]:
bridges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20415 entries, 0 to 20414
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   road              20415 non-null  object 
 1   km                20415 non-null  float64
 2   type              20415 non-null  object 
 3   LRPName           20415 non-null  object 
 4   name              20100 non-null  object 
 5   length            20406 non-null  float64
 6   condition         20415 non-null  object 
 7   structureNr       20415 non-null  int64  
 8   roadName          20415 non-null  object 
 9   chainage          20415 non-null  float64
 10  width             17408 non-null  float64
 11  constructionYear  17407 non-null  float64
 12  spans             17408 non-null  float64
 13  zone              20415 non-null  object 
 14  circle            20415 non-null  object 
 15  division          20415 non-null  object 
 16  sub-division      20415 non-null  object

In [15]:
#filter bridges with link_no hence, on relevant N roads
n_road_bridges = bridges[(bridges['link_no'].notna())]

In [16]:
n_road_bridges_with_traffic = n_road_bridges.merge(road_segments_water,'left','link_no')

In [17]:
#save to csv
n_road_bridges_with_traffic.to_csv('../data/processed/bridges_with_traffic.csv', index=False)

In [18]:
def count_bridge_types_in_segment(segment,condition):
    """Count number of bridges in segment with given condition
    Parameters
    ----------
    segment : pandas.Series
        Series containing segment data
        condition : string
            condition of bridge
    Returns
    -------
    int
        number of bridges in segment with given condition
    """
    bridges = n_road_bridges_with_traffic[(n_road_bridges_with_traffic['link_no'] == segment['link_no']) & (n_road_bridges_with_traffic['condition'] == condition)]
    return len(bridges)

In [19]:
road_segments_water['count_a_bridges'] = road_segments_water.apply(lambda segment: count_bridge_types_in_segment(segment,'A'),axis=1)
road_segments_water['count_b_bridges'] = road_segments_water.apply(lambda segment: count_bridge_types_in_segment(segment,'B'),axis=1)
road_segments_water['count_c_bridges'] = road_segments_water.apply(lambda segment: count_bridge_types_in_segment(segment,'C'),axis=1)
road_segments_water['count_d_bridges'] = road_segments_water.apply(lambda segment: count_bridge_types_in_segment(segment,'D'),axis=1)

In [20]:
def segment_sensitivity(segment):
    """ calculate sensitivity of segment
    Parameters
    ----------
    segment : pandas.Series
        Series containing segment data
    Returns
    -------
    float
        sensitivity of segment
      """
    
    return (0.19*segment['count_a_bridges']+0.44*segment['count_b_bridges']+0.71*segment['count_c_bridges']+segment['count_d_bridges'])/(segment['Chainage_End']-segment['Chainage_Start'])

In [21]:
def segment_adaptive_capacity(segment):
    """
    calculate adaptive capacity of segment
    Parameters
    ----------
    segment : pandas.Series
        Series containing segment data
    Returns
    -------
    float
        adaptive capacity of segment
    """
    
    bridges = n_road_bridges_with_traffic[(n_road_bridges_with_traffic['link_no'] == segment['link_no']) ]
    if not bridges.empty:
        #sum bridge length divided by longest bridge of network to the power of 1.2
        return sum((bridges['length']/1786)**1.2)/(segment['Chainage_End']-segment['Chainage_Start'])
    else:
        return 0
        
    

In [22]:
#calculate sensitivity and adaptive capacity
road_segments_water['sensitivity'] = road_segments_water.apply(segment_sensitivity,axis=1)
road_segments_water['adaptive_capacity'] = road_segments_water.apply(segment_adaptive_capacity,axis=1)

In [23]:
#normalize sensitivity, excess water level, and adaptive capacity
road_segments_water['sensitivity_normalized'] = (road_segments_water['sensitivity']-road_segments_water['sensitivity'].min())/(road_segments_water['sensitivity'].max()-road_segments_water['sensitivity'].min())
road_segments_water['adaptive_capacity_normalized'] = (road_segments_water['adaptive_capacity']-road_segments_water['adaptive_capacity'].min())/(road_segments_water['adaptive_capacity'].max()-road_segments_water['adaptive_capacity'].min())
road_segments_water['exposure_normalized'] = (road_segments_water['Excess Water Level']-road_segments_water['Excess Water Level'].min())/(road_segments_water['Excess Water Level'].max()-road_segments_water['Excess Water Level'].min())

In [24]:
#calculate vulnerability index
road_segments_water['vulnerability_index'] = 1/3*road_segments_water['sensitivity_normalized']+1/3*road_segments_water['adaptive_capacity_normalized']+1/3*road_segments_water['exposure_normalized']

In [25]:
#filter out segments with no vulnerability index
road_segments_water = road_segments_water[road_segments_water['vulnerability_index'].notna()]

<h1>Criticality</h1>

In [26]:
tonnage_dict = {'Motor Cycle': 0.04,
 'Auto Rickshaw': 0.17,
 'Car': 0.19,
 'Micro Bus': 0.53,
 'Medium Bus': 2.85,
 'Large Bus': 4.9,
 'Small Truck': 5.0,
 'Medium Truck': 14.0,
 'Heavy Truck': 28.0,
 'Utility': 0.31,
 'Cycle Rickshaw': 0.1,
 'Bi-Cycle': 0.02,
 'Cart': 0.15}

In [27]:
# define a function to calculate total tonnage for each row based on count and values
def calculate_tonnage(row):
    """calculate total tonnage for each row based on count and values
    Parameters
    ----------
    segment : pandas.Series
        Series containing segment data
    Returns
    -------
    float
       total tonnage for each row based on count and values of tonnage_dict
    
    """
    
    total_tonnage = 0
    for mode, count in row.items():
         if mode in tonnage_dict:
            total_tonnage += count * tonnage_dict[mode]
    return total_tonnage

In [28]:
road_segments_water_tonnage = road_segments_water

In [29]:
#calculate total tonnage for each row based on count and values of tonnage_dict
road_segments_water_tonnage['total_tonnage'] = road_segments_water.apply(calculate_tonnage, axis=1)

In [30]:
# calculate average tonnage per road name and save it in a new column 'avg_tonnage'
road_segments_water_tonnage['avg_tonnage_road'] = road_segments_water_tonnage.groupby('road_name')['total_tonnage'].transform('mean')
#weighted total tonnage
road_segments_water_tonnage['weighted_total_tonnage'] = road_segments_water_tonnage['total_tonnage']/road_segments_water_tonnage['avg_tonnage_road']

In [31]:
#normalize weighted total tonnage in criticality
min_value = road_segments_water_tonnage['weighted_total_tonnage'].min()
max_value = road_segments_water_tonnage['weighted_total_tonnage'].max()
road_segments_water_tonnage['criticality_index'] = (road_segments_water_tonnage['weighted_total_tonnage'] - min_value) / (max_value - min_value)

In [32]:
#importance index as average of vulnerability and criticality
road_segments_water_tonnage['importance_index'] = (road_segments_water_tonnage['vulnerability_index']+road_segments_water_tonnage['criticality_index'])/2

In [33]:
#save data to processed folder as csv
road_segments_water_tonnage.to_csv('../data/processed/processed_road_segments.csv',index=False)
